# Clustering based on **Curve fitting**

Data (chart type) clustering based on curve fitting. Fit different functions:

- first- to six-order **_polynomial_**
- **_quadratic_**
- **_square root_**
- **_exponential_**
- **_logarithmic_**
- **_reciprocal 1/x_**
- **_sinus_**
- **_cosine_**
- **_Gaussian_**

In [ ]:
import boto3
from sagemaker import get_execution_role
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
import warnings

warnings.simplefilter('ignore')

Objectives functions

In [ ]:
def obj_quadratic(x, a):
    return a * (x**2)

def obj_quad(x, a, b):
    return -(a * x + b*x*x)

def obj_square(x, a):
    return a * np.sqrt(x)

def obj_exponential(x, a, b, c):
    return a * np.exp(-b * x) + c

def obj_log(x, a, b, c):
    return a * np.log(b * x) + c

# reciprocal 1 / x function
def obj_divide(x, a, b, c, d):
    return c * (1 / (b * x - d)) 

def obj_sin(x, a, b, c, d):
    return a * np.sin(b - x) + c * x**2 + d

def obj_cos(x, Amp, n, b):
    return Amp*(np.abs(np.cos(x)))**n + b

def obj_gaus(x, a, x0, sigma):
    n = len(x)
    mean = sum(x * y) / n  # note this correction
    sigma = sum(y * (x - mean)**2) / n # note this correction
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

Curve fitting functions

In [ ]:
def fit_quadratic(x, y):
    try: 
        popt, _ = curve_fit(obj_quadratic, x[:-1], y)
        # summarize the parameter values
        a = popt
        # define a sequence of inputs between the smallest and largest known inputs
        x_line = np.arange(min(x), max(x), 1)
        # calculate the output for the range
        y_line = obj_quadratic(x_line, a)
    except:
        y_line = 100
    return y_line

def fit_quad(x, y):
    try:
        popt, _ = curve_fit(obj_quad, x[:-1], y)
        a, b = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_quad(x_line, a, b)
    except:
        y_line = 100
    return y_line

def fit_square(x, y):
    try:
        popt, _ = curve_fit(obj_square, x[:-1], y)
        a = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_square(x_line, a)
    except:
        y_line = 100
    return y_line

def fit_exponential(x, y):
    try:
        popt, _ = curve_fit(obj_exponential, x[:-1], y)
        a, b, c = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_exponential(x_line, a, b, c)
    except:
        y_line = 100
    return y_line

def fit_log(x, y):
    try:
        popt, _ = curve_fit(obj_log, x[:-1], y)
        a, b, c = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_log(x_line, a, b, c)
    except:
        y_line = 100
    return y_line

def fit_divide(x, y):
    try:
        popt, _ = curve_fit(obj_divide, x[:-1], y)
        a, b, c, d = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_divide(x_line, a, b, c, d)
    except:
        y_line = 100
    return y_line

def fit_sin(x, y):
    try:
        popt, _ = curve_fit(obj_sin, x[:-1], y)
        a, b, c, d = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_sin(x_line, a, b, c, d)
    except:
        y_line = 100
    return y_line

def fit_cos(x, y):
    try:
        popt, _ = curve_fit(obj_cos, x[:-1], y)
        Amp, n, b = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_cos(x_line, Amp, n, b)
    except:
        y_line = 100
    return y_line

def fit_gaus(x, y):
    try:
        popt, _ = curve_fit(obj_gaus, x[:-1], y)
        #popt, _ = curve_fit(gaus, x, y, p0=[1,mean,sigma])
        a, x0, sigma = popt
        x_line = np.arange(min(x), max(x), 1)
        y_line = obj_gaus(x_line, a, x0, sigma)
    except:
        y_line = 100
    return y_line

## Residuals

> These deviations are called **residuals** when the calculations are performed over the data sample that was used for estimation.

### **R-squared**

> for **_Linear regression_** only

In [ ]:
def r_squared(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    # r-squared
    p = np.poly1d(coeffs)
    # fit values, and mean
    yhat = p(x)                      # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    results['r_squared'] = ssreg / sstot

    return results

### Define function to calculate **adjusted R-squared** 
> for **_Linear regression_** only

In [ ]:
def adj_R(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    p = np.poly1d(coeffs)
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['adjusted_r_squared'] = 1- (((1-(ssreg/sstot))*(len(y)-1))/(len(y)-degree-1))

    return results

In [ ]:
print(adj_R(x, y, 5))
print(r_squared(x, y, 5))
#adjR(x, y, quadratic_model)

### Root-Mean-Square Deviation (RMSD) or Root-Mean-Square Error (RMSE)

In [ ]:
def rmse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.sqrt(np.square(np.subtract(actual,pred)).mean())

### Standart Error of the Mean

In [ ]:
from scipy.stats import sem

#define dataset 
data = []

#calculate standard error of the mean 
sem(data)

### Mean Squared Error (MSE)

In [ ]:
def mse(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.square(np.subtract(actual,pred)).mean() 

## Clustering based on **Curve fitting**

1. Curve fitting
2. Calculating Root-Mean-Square Error (RMSE) for each function (polynomial, quadratic, square root, etc.)

In [ ]:
def curve_fitting(x, y, index):
    
    '''
    Parameters:
        x (list): input variable
        y (list): output variable
        index (int): row index
    
    Returns:
        df_curve: new dataframe with all values for 
    '''
    
    
    # curve fitting with all types of models and calculate Root-Mean-Square Error (RMSE)
    
    df_curve.at[index, 'model1'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 1), x[:-1])).astype(float) # first-order polynomial
    df_curve.at[index, 'model2'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 2), x[:-1])).astype(float) # second-order polynomial
    df_curve.at[index, 'model3'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 3), x[:-1])).astype(float) # third-order polynomial
    df_curve.at[index, 'model4'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 4), x[:-1])).astype(float) # fourth-order polynomial
    df_curve.at[index, 'model5'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 5), x[:-1])).astype(float) # fifth-order polynomial
    df_curve.at[index, 'model6'] = rmse(y, np.polyval(np.polyfit(x[:-1], y, 6), x[:-1])).astype(float) # six-order polynomial
    
    df_curve.at[index, 'quadratic_model'] = rmse(y, fit_quadratic(x, y)).astype(float) # quadratic
    df_curve.at[index, 'quad_model'] = rmse(y, fit_quad(x, y)).astype(float) # quadratic
    df_curve.at[index, 'square_model'] = rmse(y, fit_square(x, y)).astype(float) # square root
    
    #df_curve.at[index, 'exp_model'] = rmse(y, curve_fit(lambda t,a,b: a * np.exp(b * t),  x[:-1],  y)) # exponential
    df_curve.at[index, 'exp_model_1'] = rmse(y, fit_exponential(x, y)).astype(float) # exponential
    
    #df_curve.at[index, 'log_model'] = rmse(y, curve_fit(lambda t,a,b: a + b * np.log(t),  x[:-1],  y)) # logarithmic
    df_curve.at[index, 'log_model_1'] = rmse(y, fit_log(x, y)).astype(float) # logarithmic
    
    df_curve.at[index, 'div_model'] = rmse(y, fit_divide(x, y)).astype(float) # reciprocal 1/x
    
    df_curve.at[index, 'sin_model'] = rmse(y, fit_sin(x,y)).astype(float) # sinus
    df_curve.at[index, 'cos_model'] = rmse(y, fit_cos(x, y)).astype(float)# cosine
    
    df_curve.at[index, 'gaus_model'] = rmse(y, fit_gaus(x, y)).astype(float) # Gaussian 
    
    
    return df_curve 

3. Find a curve that fit the best to each chart (finding the lowest RMSE value)

In [ ]:
def min_value(df):
    df_curve.apply(pd.to_numeric)
    df_curve['curve'] = df_curve.idxmin(axis=1)
    return df_curve

# 

--------------------------

In [ ]:
role = get_execution_role()
bucket='path-to/'
data_key = 'file.csv' 
data_location = 's3://{}/{}'.format(bucket, data_key)

In [ ]:
df = pd.DataFrame(pd.read_csv(data_location))
df = df[['column_name', 'column_name']]

#print(df.shape)
#df.head(2)

In [ ]:
# choose the input and output variables
x = df.columns[1:].astype(int).tolist()
#df_curve = df.copy(deep=True)

df_curve = pd.DataFrame()

for index, row in df.iterrows():
    y = df.loc[index, df.columns[1:-1]].astype(float).tolist()
    curve_fitting(x, y, index)
    
min_value(df_curve)

In [ ]:
df_final = pd.DataFrame()
df_final = df_curve['curve']

df.reset_index(drop=True, inplace=True)
df_final.reset_index(drop=True, inplace=True)

df_new = pd.concat([df, df_final], axis=1)

In [ ]:
df_new.head(5)

In [ ]:
df_new['curve'].value_counts().plot.bar(figsize=(16, 9))

In [ ]:
print(df_new['curve'].value_counts())

# 
____________________________________________

# Curve fitting for a single item

In [ ]:
# choose the input and output variables
x = df.columns[1:-1].astype(int).tolist()
y = df.loc[2, df.columns[1:-1]].astype(float).tolist()

## Logarithmic function fitting

In [ ]:
log_model = np.poly1d(np.polyfit(x, np.log(y), 1, w=np.sqrt(y)))

#create scatterplot
plt.figure(figsize=(14,7))
plt.scatter(x, y)

#add fitted polynomial lines to scatterplot 
plt.plot(x, log_model(x), color='green', label='1')

## Exponential function fitting

In [ ]:
#yn = y + 0.2*np.random.normal(size=len(x))

popt, _ = curve_fit(obj_exponential, x, y)
# summarize the parameter values
a, b, c = popt
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = obj_exponential(x_line, a, b, c)

# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Reciprocal 1/x function fitting

In [ ]:
popt, _ = curve_fit(obj_divide, x, y)
# summarize the parameter values
a, b, c, d = popt
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = obj_divide(x_line, a, b, c, d)

# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Square function fitting

In [ ]:
popt, _ = curve_fit(obj_square, x, y)
# summarize the parameter values
a = popt
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = obj_square(x_line, a)

# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Quadratic function fitting

In [ ]:
popt, _ = curve_fit(obj_quadratic, x, y)
# summarize the parameter values
a = popt
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = obj_quadratic(x_line, a)

# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Polynomial function fitting

In [ ]:
# define the true objective function
def objective(x, a, b, c, d, e, f):
    return (a * x) + (b * x**2) + (c * x**3) + (d * x**4) + (e * x**5) + f
 
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c, d, e, f = popt
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c, d, e, f)
# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Sinus function fitting

In [ ]:
# define the true objective function
def objective(x, a, b, c, d):
	return a * np.sin(b - x) + c * x**2 + d
 
# curve fit
popt, _ = curve_fit(objective, x, y)
# summarize the parameter values
a, b, c, d = popt
print(popt)
# define a sequence of inputs between the smallest and largest known inputs
x_line = arange(min(x), max(x), 1)
# calculate the output for the range
y_line = objective(x_line, a, b, c, d)
# plot input vs output
plt.figure(figsize=(14,7))
plt.scatter(x, y)
# create a line plot for the mapping function
plt.plot(x_line, y_line, '--', color='red')
plt.legend()
plt.show()

## Polynomial function fitting up to six-order

In [ ]:
#fit polynomial models up to degree 6
model1 = np.poly1d(np.polyfit(x, y, 1))
model2 = np.poly1d(np.polyfit(x, y, 2))
model3 = np.poly1d(np.polyfit(x, y, 3))
model4 = np.poly1d(np.polyfit(x, y, 4))
model5 = np.poly1d(np.polyfit(x, y, 5))
model6 = np.poly1d(np.polyfit(x, y, 6))


#create scatterplot
plt.figure(figsize=(14,7))
plt.scatter(x, y)

#add fitted polynomial lines to scatterplot 
plt.plot(x, model1(x), color='green', label='1')
plt.plot(x, model2(x), color='red', label='2')
plt.plot(x, model3(x), color='purple', label='3')
plt.plot(x, model4(x), color='blue', label='4')
plt.plot(x, model5(x), color='orange', label='5')
plt.plot(x, model6(x), color='brown', label='6')
plt.legend()
plt.show()

###  Calculate adjusted R-squared for polynomial functions

In [ ]:
#define function to calculate adjusted r-squared
def adjR(x, y, degree):
    results = {}
    coeffs = np.polyfit(x, y, degree)
    p = np.poly1d(coeffs)
    yhat = p(x)
    ybar = np.sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)
    sstot = np.sum((y - ybar)**2)
    results['r_squared'] = 1- (((1-(ssreg/sstot))*(len(y)-1))/(len(y)-degree-1))
    print(results)

    return results

#calculated adjusted R-squared of each model
adjR(x, y, 1)
adjR(x, y, 2)
adjR(x, y, 3)
adjR(x, y, 4)
adjR(x, y, 5)
adjR(x, y, 6)